In [30]:
import pandas as pd
from spacy.tokenizer import Tokenizer
from spacy.lang.en import English
import statistics
from scipy import spatial, stats
import numpy as np
from statistics import mean
from scipy.stats import ttest_ind
import os

Read in the meta data an theta matrix in dataframes and reset their indices.

In [31]:
def find(name, path):
    for root, dirs, files in os.walk(path):
        if name in files:
            return os.path.join(root, name)

In [2]:
path = find('meta.csv','C://')
data = pd.read_csv(path)
path = find('thetamatrix50.csv', 'C://')
theta = pd.read_csv(path)
data = data.reset_index(drop=True)
theta = theta.reset_index(drop=True)

C:\Users\20150624\AppData\Local\Continuum\anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (23,32) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Create a dataframe that reports for every pair of targets and acquirers the median topic proportions over all their documents, if the document was before or after the effective date, the similarity between the target and acquirer vector and their industries.

In [3]:
proportions = pd.DataFrame(columns=['Target', 'Acquiror', 'TargetTopics', 'AcquirorTopics', 'Period', 'Similarity'])
nlp = English()
tokenizer = Tokenizer(nlp.vocab)
target_text_length = list()
acquirer_text_length = list()
before = list()
after = list()
valid_pair_before = list()
valid_pair_after = list()
same_industry = list()
diff_industry = list()
for company in data['companydate'].unique():
    target = list(data[data['companydate']==company]['Target.Name'])[0]
    acquirer = list(data[data['companydate']==company]['Acquiror.Name'])[0]
    target_industry = list(data[data['Target.Name']==target]['Target.Macro.Industry'])[0]
    acquiror_industry = list(data[data['Acquiror.Name']==acquirer]['Acquiror.Macro.Industry'])[0]    
    year = list(data[data['companydate']==company]['DealYear'])[0]
    docyear = list(data[data['companydate']==company]['DocumentYear'])[0]
    pair = (target, acquirer)
    period =list(data[data['companydate']==company]['period'])[0]
    target_topics= list()
    acquiror_topics = list()
    if (period == 'before') and (pair not in before):
        before.append(pair)
        indices = data[(data['Target.Name']==target)&(data['Acquiror.Name']==acquirer)&(data['period']=='before')].index
        target_indices = list()
        acquirer_indices = list()
        target_length = 0
        acquirer_length = 0
        for index in indices:
            if data.iloc[index]['companytype'] == 'Target':
                target_indices.append(index)
            if data.iloc[index]['companytype'] ==  'Acquiror':
                acquirer_indices.append(index)
        for index in target_indices:
            text = data.iloc[index]['clean_text']
            tokens = tokenizer(text)
            target_length += len(tokens)
        for index in acquirer_indices:
            text = data.iloc[index]['clean_text']
            tokens = tokenizer(text)
            acquirer_length += len(tokens)
        target_text_length.append(target_length)
        acquirer_text_length.append(acquirer_length)
        if target_length >=162 and acquirer_length >=162:
            valid_pair_before.append(pair)
            for column in theta.columns:
                if column != 'Unnamed: 0':
                    target_topics.append(theta.iloc[target_indices][column].median())
                    acquiror_topics.append(theta.iloc[acquirer_indices][column].median())
            sum_target_topics=sum(target_topics)
            target_topics = [i/sum_target_topics for i in target_topics]
            sum_acquiror_topics = sum(acquiror_topics)
            acquiror_topics = [i/sum_acquiror_topics for i in acquiror_topics]
            similarity = 1-spatial.distance.euclidean(target_topics, acquiror_topics)
            if target_industry == acquiror_industry:
                same_industry.append(similarity)
            if target_industry != acquiror_industry:
                diff_industry.append(similarity)
            props = pd.DataFrame({'Target':target,'Acquiror':acquirer,'TargetTopics':[target_topics],'AcquirorTopics':[acquiror_topics],'Period':period,
                                 'Target industry':target_industry,'Acquirer industry':acquiror_industry,
                                 'Similarity':similarity,'Year':int(year), 'DocYear':int(docyear)})
            proportions = proportions.append(props)
    if (period == 'after') and (pair not in after):
        after.append(pair)
        indices = data[(data['Target.Name']==target)&(data['Acquiror.Name']==acquirer)&(data['period']=='after')].index
        target_indices = list()
        acquirer_indices = list()
        target_length = 0
        acquirer_length = 0
        for index in indices:
                if data.iloc[index]['companytype'] == 'Target':
                    target_indices.append(index)
                if data.iloc[index]['companytype'] ==  'Acquiror':
                    acquirer_indices.append(index)
        for index in target_indices:
            text = data.iloc[index]['clean_text']
            tokens = tokenizer(text)
            target_length += len(tokens)
        for index in acquirer_indices:
            text = data.iloc[index]['clean_text']
            tokens = tokenizer(text)
            acquirer_length += len(tokens)
        target_text_length.append(target_length)
        acquirer_text_length.append(acquirer_length)
        if target_length >=162 and acquirer_length >=162:
            valid_pair_after.append(pair)
            for column in theta.columns:
                if column != 'Unnamed: 0':
                    target_topics.append(theta.iloc[target_indices][column].median())
                    acquiror_topics.append(theta.iloc[acquirer_indices][column].median())
            sum_target_topics=sum(target_topics)
            target_topics = [i/sum_target_topics for i in target_topics]
            sum_acquiror_topics = sum(acquiror_topics)
            acquiror_topics = [i/sum_acquiror_topics for i in acquiror_topics]                    
            similarity = 1-spatial.distance.euclidean(target_topics, acquiror_topics)
            if target_industry == acquiror_industry:
                same_industry.append(similarity)
            if target_industry != acquiror_industry:
                diff_industry.append(similarity)
            props = pd.DataFrame({'Target':target,'Acquiror':acquirer,'TargetTopics':[target_topics],'AcquirorTopics':[acquiror_topics],'Period':period,
                                 'Target industry':target_industry,'Acquirer industry':acquiror_industry,
                                  'Similarity':similarity, 'Year':int(year),'DocYear':int(docyear)})
            proportions = proportions.append(props)
proportions = proportions.reset_index(drop=True)

C:\Users\20150624\AppData\Local\Continuum\anaconda3\lib\site-packages\pandas\core\frame.py:6211: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [48]:
proportions

,Acquirer industry,Acquiror,AcquirorTopics,DocYear,Period,Similarity,Target,Target industry,TargetTopics,Year
0,Healthcare,Absolute Total Care,"[0.00904475435754733, 0.003345501720923122, 0....",2010.0,before,0.861673,Carolina Crescent Health Plan,Healthcare,"[0.005597960145786435, 0.0021563806692588613, ...",2010.0
1,Healthcare,Centene Corp,"[0.014813685662987925, 0.004903738188395357, 0...",2013.0,before,0.607572,Specialty Therapeutic Hldg Inc,Retail,"[0.00695398434468514, 0.0002217933282976976, 0...",2013.0
2,Healthcare,DentaQuest LLC,"[0.002163036116416527, 0.00045437753787188386,...",2017.0,before,0.371596,Access Dental Plan LLC,Healthcare,"[0.0022298939174939247, 0.00031808298823092935...",2017.0
3,Healthcare,Tufts Assocd Health Plans Inc,"[0.0106516271344824, 0.011384715648700028, 0.0...",2015.0,before,0.680485,Integra Partners LLC,Financials,"[0.0050995620356424284, 0.005302140636081216, ...",2015.0
4,Healthcare,Aetna Inc,"[0.008960587393831272, 0.006458270350028384, 0...",2011.0,before,0.592219,Healthagen LLC,High Technology,"[0.006498347116312149, 0.001489436368894553, 0...",2011.0
5,Healthcare,WellCare Health Plans Inc,"[0.004456534518803531, 0.0006543031554535112, ...",2013.0,before,0.678634,Aetna Inc-Missouri Medicaid,Healthcare,"[0.005906873304121198, 0.00038608073440771614,...",2013.0
6,Healthcare,Aetna Inc,"[0.0022821447647642787, 0.00035717995789639, 0...",2013.0,before,0.426120,Coventry Health Care Inc,Healthcare,"[0.008774250680172856, 0.002253050586413694, 0...",2013.0
7,Healthcare,Aetna Inc,"[0.002011055297666133, 0.0031400090710780857, ...",2014.0,before,0.603189,bswift LLC,High Technology,"[0.0009751831617086384, 0.00588683912799735, 0...",2014.0
8,Healthcare,UnitedHealth Group Inc,"[0.0006315746781651798, 0.0047417768385622835,...",2017.0,before,0.216138,American Health Network Inc,Healthcare,"[0.036790834455616296, 0.0030046989047821126, ...",2017.0
9,Financials,GIG,"[0.0005204436293681085, 0.00416549580096611, 0...",2017.0,before,0.750701,AIG Sigorta AS,Healthcare,"[0.001831749739083088, 0.01496709131570686, 0....",2017.0


In [29]:
d = dict.fromkeys(np.arange(1,51,1))
l = list(proportions[proportions['Acquiror']=='Gravie Inc']['AcquirorTopics'])[0]
for i in l:
    d[l.index(i)+1] = i
{k: v for k, v in sorted(d.items(), key=lambda item: item[1], reverse=True)}


{27: 0.3675931619012694,
 13: 0.1818899170135936,
 45: 0.14372678599687894,
 50: 0.08362555938774983,
 6: 0.04248821747306464,
 31: 0.033291535341144554,
 3: 0.02615412419875429,
 22: 0.022563126463641843,
 16: 0.015031519252856578,
 33: 0.014097239136201352,
 36: 0.009529389905728882,
 32: 0.005003938300776443,
 37: 0.00460641504373706,
 4: 0.003923141735289841,
 8: 0.003717430903598743,
 28: 0.003132816132209819,
 12: 0.00295617568707989,
 20: 0.002807370871836439,
 34: 0.0027882376886931405,
 43: 0.0027543114551493186,
 44: 0.002236780200583561,
 15: 0.0020791583354325127,
 29: 0.0020433577856190255,
 21: 0.0019122351295822978,
 2: 0.0016838957182331706,
 24: 0.0015814811548830014,
 38: 0.0015628840187990748,
 47: 0.00141741933126695,
 30: 0.001295041636050524,
 49: 0.0009837960774360867,
 17: 0.000952551927627259,
 41: 0.0009345185651747402,
 5: 0.0008518508664289437,
 35: 0.0007754697057828822,
 11: 0.0007595749862534431,
 26: 0.0007489686482304262,
 39: 0.0007052845097679033,
 25

Read in the correlations between topics and drop an unnecessary column.

In [4]:
path = find('positivecorrelationstopics.csv', 'C://')
relations = pd.read_csv(path)
relations = relations.drop(labels='Unnamed: 0', axis=1)

In [5]:
relations

,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V41,V42,V43,V44,V45,V46,V47,V48,V49,V50
0,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,...,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
1,0.000000,1.000000,0.067571,0.000000,0.000000,0.000000,0,0.019234,0.000000,0.000000,...,0.000000,0,0.000000,0.048835,0.018212,0.000000,0.000000,0.000000,0.000000,0.000000
2,0.000000,0.067571,1.000000,0.000000,0.000000,0.000000,0,0.084470,0.000000,0.000000,...,0.000000,0,0.000000,0.244040,0.072299,0.031512,0.000000,0.000000,0.000000,0.000000
3,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0,0.000000,0.000000,0.000000,...,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.089891
4,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0,0.000000,0.000000,0.000000,...,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
5,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0,0.000000,0.000000,0.000000,...,0.000000,0,0.000000,0.000000,0.024294,0.000000,0.000000,0.000000,0.000000,0.000000
6,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1,0.000000,0.000000,0.000000,...,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
7,0.000000,0.019234,0.084470,0.000000,0.000000,0.000000,0,1.000000,0.000000,0.000000,...,0.000000,0,0.011836,0.015943,0.000000,0.010162,0.000000,0.000000,0.000000,0.000000
8,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,1.000000,0.000000,...,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
9,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0,0.000000,0.000000,1.000000,...,0.000000,0,0.000000,0.000000,0.000000,0.000000,0.047650,0.000000,0.000000,0.000000


Compute the complementarity between a target and acquirer by iterating over the rows in the dataframe. Complementarity is based on the topic proportions and relation between the topics. 

In [6]:
complementarities = pd.DataFrame(columns = ['Target', 'Acquirer', 'TargetTopics', 'AcquirerTopics','Complementarity', 'Period'])
for index, row in proportions.iterrows():
    complementarity = 0
    acquirer_topics = row['AcquirorTopics']
    target_topics = row['TargetTopics']
    for topic in acquirer_topics:
        index=acquirer_topics.index(topic)
        correlations = relations.iloc[acquirer_topics.index(topic)]
        for topic2 in target_topics:
            index2 = target_topics.index(topic2)
            #Check if not the same topic
            if index != index2:
                targetforthattopic = target_topics[index]
                acquirerforothertopic = acquirer_topics[index2]
                correlation = correlations[target_topics.index(topic2)]
                #Check if both companies have the largest proportion for one of the topics
                if topic > targetforthattopic and acquirerforothertopic < topic2:
                    complementarity += topic*correlation + topic2*correlation
                elif targetforthattopic > topic and acquirerforothertopic > topic2:
                    complementarity += targetforthattopic*correlation + acquirerforothertopic*correlation
    if row['Target industry'] == row['Acquirer industry']:
        ind = 'Same'
    elif row['Target industry'] != row['Acquirer industry']:
        ind = 'Different'
    complement = pd.DataFrame({'Target':row['Target'],'TargetIndustry':row['Target industry'],'Acquirer':row['Acquiror'],'TargetTopics':[target_topics],
                               'AcquirerIndustry':row['Acquirer industry'],'AcquirerTopics':[acquirer_topics],
                               'Complementarity':complementarity, 'Period':row['Period'], 'Industry':ind})
    complementarities = complementarities.append(complement)
complementarities = complementarities.reset_index(drop=True)    

Compute the complementarity between a target and the mean of all other acquirers and an acquirer and the mean of all other targets. 

In [7]:
other_complementarities = pd.DataFrame(columns=['Target', 'Acquirer', 'Period', 'Complementarity', 'Mean Complementarity Other Acquirers', 'Mean Complementarity Other Targets'])
complementarity_highest = 0
other_acquirers_higher =0
other_targets_higher = 0
for index, row in complementarities.iterrows():
    other_com_target = list()
    other_com_acquirer = list()
    complementarity = row['Complementarity']
    acquirer_topics = row['AcquirerTopics']
    target_topics = row['TargetTopics']
    period = row['Period']
    for index2, row2 in complementarities.iterrows():
        if period == row2['Period']:
            ttcomplementarity = 0
            atcomplementarity = 0
            if index != index2:
                at = row2['AcquirerTopics']
                tt = row2['TargetTopics']
                for topic in acquirer_topics:
                    correlations = relations.iloc[acquirer_topics.index(topic)]
                    for topic2 in tt:
                        i1 = acquirer_topics.index(topic)
                        i2 = tt.index(topic2)
                        if i1 != i2:
                            correlation = correlations[tt.index(topic2)]
                            targetforthattopic = tt[i1]
                            acquirerforthattopic = acquirer_topics[i2]
                            if topic > targetforthattopic and topic2 > acquirerforthattopic:
                                ttcomplementarity += topic*correlation + topic2*correlation
                            elif targetforthattopic > topic and acquirerforthattopic >topic2:
                                ttcomplementarity += topic*correlation + topic2*correlation
                for topic in target_topics:
                    correlations = relations.iloc[target_topics.index(topic)]
                    for topic2 in at:
                        i1 = target_topics.index(topic)
                        i2 = at.index(topic2)
                        if i1 != i2:
                            correlation = correlations[at.index(topic2)]
                            acquirerforthattopic = at[i1]
                            targetforthattopic = target_topics[i2]
                            if topic > acquirerforthattopic and topic2 > targetforthattopic:
                                atcomplementarity += topic*correlation + topic2*correlation
                            elif acquirerforthattopic > topic and targetforthattopic >topic2:
                                atcomplementarity += topic*correlation + topic2*correlation                        
            other_com_target.append(ttcomplementarity)
            other_com_acquirer.append(atcomplementarity)
    complement = pd.DataFrame({'Target':row['Target'],'TargetIndustry':row['TargetIndustry'], 'Acquirer': row['Acquirer'],'Period':row['Period'],
                               'AcquirerIndustry':row['AcquirerIndustry'],'Complementarity':complementarity,'Mean Complementarity Other Acquirers':mean(other_com_acquirer),
                              'Mean Complementarity Other Targets':mean(other_com_target),'Industry':row['Industry']}, index=[index])
    other_complementarities = other_complementarities.append(complement)

In [8]:
other_complementarities

,Acquirer,AcquirerIndustry,Complementarity,Industry,Mean Complementarity Other Acquirers,Mean Complementarity Other Targets,Period,Target,TargetIndustry
0,Absolute Total Care,Healthcare,0.516205,Same,0.213777,0.223259,before,Carolina Crescent Health Plan,Healthcare
1,Centene Corp,Healthcare,0.761421,Different,0.356706,0.244007,before,Specialty Therapeutic Hldg Inc,Retail
2,DentaQuest LLC,Healthcare,0.364788,Same,0.253879,0.096471,before,Access Dental Plan LLC,Healthcare
3,Tufts Assocd Health Plans Inc,Healthcare,0.617080,Different,0.266653,0.223335,before,Integra Partners LLC,Financials
4,Aetna Inc,Healthcare,0.365080,Different,0.200812,0.223231,before,Healthagen LLC,High Technology
5,WellCare Health Plans Inc,Healthcare,0.388378,Same,0.160232,0.181384,before,Aetna Inc-Missouri Medicaid,Healthcare
6,Aetna Inc,Healthcare,0.306261,Same,0.211286,0.123733,before,Coventry Health Care Inc,Healthcare
7,Aetna Inc,Healthcare,0.272360,Different,0.276743,0.168911,before,bswift LLC,High Technology
8,UnitedHealth Group Inc,Healthcare,0.231711,Same,0.167111,0.171939,before,American Health Network Inc,Healthcare
9,GIG,Financials,0.596959,Different,0.234572,0.213263,before,AIG Sigorta AS,Healthcare


T-test to check if the complementarity between deals with companies from the same industry significanlty differs from deals with companies from different industries.

In [9]:
complementarities_same = other_complementarities[other_complementarities['Industry']=='Same']
complementarities_diff = other_complementarities[other_complementarities['Industry']=='Different']
ttest_ind(complementarities_same['Complementarity'], complementarities_diff['Complementarity'])

Ttest_indResult(statistic=-0.11070518095068238, pvalue=0.9119579184586895)

In [10]:
print(complementarities_same['Complementarity'].mean())
print(complementarities_diff['Complementarity'].mean())

0.37290429311616
0.37560759006399436


In [79]:
print(other_complementarities['Complementarity'].mean())
print(other_complementarities['Mean Complementarity Other Acquirers'].mean())
print(other_complementarities['Mean Complementarity Other Targets'].mean())

0.35607824509960406
0.20931309586897448
0.21381145290270892
0.37419095849036965
0.20853794461050257
0.20853794461050257


Count how often the complementarity between a target and its acquirer were the highest or how often the mean of other acquirers or targets were higher.

In [66]:
complementarity_highest = 0
acquirer_higher = 0
target_higher = 0
for index, row in other_complementarities.iterrows():
    c = row['Complementarity']
    a = row['Mean Complementarity Other Targets']
    t = row['Mean Complementarity Other Acquirers']
    if c > a and c >t:
        complementarity_highest +=1
    if a > c:
        acquirer_higher +=1
    if t > c:
        target_higher +=1
print(complementarity_highest)
print(acquirer_higher)
print(target_higher)

168
30
29


In [76]:
other_complementarities.to_csv('C:/complementaritiesresults.csv')

T-test to check if the mean complementarity between a target and acquirer was significantly higher than the mean complementarity of a target with all other acquirers.

In [80]:
ttest_ind(other_complementarities['Complementarity'], other_complementarities['Mean Complementarity Other Acquirers'])

Ttest_indResult(statistic=13.134179431831962, pvalue=3.4914803892353054e-33)

T-test to check if the mean complementarity between a target and acquirer was significantly higher than the mean complementarity of an acquirers with all other targets.

In [81]:
ttest_ind(other_complementarities['Complementarity'], other_complementarities['Mean Complementarity Other Targets'])

Ttest_indResult(statistic=13.123414901343146, pvalue=3.8600397854368914e-33)

Subset of dataframes with only the instances before the deal or only after.

In [82]:
complementarities_before = complementarities[complementarities['Period']=='before']
complementarities_after = complementarities[complementarities['Period']=='after']

Create a dataframe with deals with complete data before and after the effective date.

In [83]:
complementarities_time = pd.DataFrame(columns=['Target', 'Acquirer', 'Before', 'After'])
for index, row in complementarities_before.iterrows():
    target = row['Target']
    acquirer = row['Acquirer']
    before = row['Complementarity']
    if len(complementarities_after[(complementarities_after['Target']==target) & (complementarities_after['Acquirer']==acquirer)])==1:
        after = complementarities_after[(complementarities_after['Target']==target) & (complementarities_after['Acquirer']==acquirer)]['Complementarity'].values[0]
        complement = pd.DataFrame({'Target':target, 'Acquirer':acquirer, 'Before':before,'After':after}, index=[index])
        complementarities_time = complementarities_time.append(complement)

In [84]:
complementarities_time

,Target,Acquirer,Before,After
1,Specialty Therapeutic Hldg Inc,Centene Corp,0.761421,0.656065
3,Integra Partners LLC,Tufts Assocd Health Plans Inc,0.617080,0.563758
4,Healthagen LLC,Aetna Inc,0.365080,0.184230
5,Aetna Inc-Missouri Medicaid,WellCare Health Plans Inc,0.388378,0.405381
6,Coventry Health Care Inc,Aetna Inc,0.306261,0.356722
8,American Health Network Inc,UnitedHealth Group Inc,0.231711,0.217260
9,AIG Sigorta AS,GIG,0.596959,0.538150
12,All Points EAP & Orgal Svcs,AllOne Health Resource Inc,0.508213,0.734080
13,Amerigroup Virginia Inc,Inova Health System,0.250749,0.310250
15,Sec Life Ins Co of America,Ameritas Life Insurance Corp,0.222532,0.110797


In [85]:
print(complementarities_time['Before'].mean())
print(complementarities_time['After'].mean())

0.36127274302662
0.36899556478576195


T-test to check if the complementarity before was significantly different from the complementarity after the effective date.

In [86]:
ttest_ind(complementarities_time['Before'], complementarities_time['After'])

Ttest_indResult(statistic=-0.2450328393148533, pvalue=0.8068875401611535)

In [ ]:
complementarities_time.to_csv('C:/complementaritiestimeresults.csv')

In [87]:
complementarities_time

,Target,Acquirer,Before,After
1,Specialty Therapeutic Hldg Inc,Centene Corp,0.761421,0.656065
3,Integra Partners LLC,Tufts Assocd Health Plans Inc,0.617080,0.563758
4,Healthagen LLC,Aetna Inc,0.365080,0.184230
5,Aetna Inc-Missouri Medicaid,WellCare Health Plans Inc,0.388378,0.405381
6,Coventry Health Care Inc,Aetna Inc,0.306261,0.356722
8,American Health Network Inc,UnitedHealth Group Inc,0.231711,0.217260
9,AIG Sigorta AS,GIG,0.596959,0.538150
12,All Points EAP & Orgal Svcs,AllOne Health Resource Inc,0.508213,0.734080
13,Amerigroup Virginia Inc,Inova Health System,0.250749,0.310250
15,Sec Life Ins Co of America,Ameritas Life Insurance Corp,0.222532,0.110797
